# Accessing ITS_LIVE data via S3 bucket
- looks like some (at least 1?) HMA granule is working now
- let's try to explore the different 32645 granules, see what is available and what their spatial coverage/relationship is

In [1]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
from shapely.geometry import Polygon
import cartopy.crs as ccrs



## Function that we'll use later to see spatial extent of different zarr objects

In [2]:
def get_bbox(input_xr):

    xmin = input_xr.coords['x'].data.min()
    xmax = input_xr.coords['x'].data.max()

    ymin = input_xr.coords['y'].data.min()
    ymax = input_xr.coords['y'].data.max()

    pts_ls = [(xmin, ymin), (xmax, ymin),(xmax, ymax), (xmin, ymax), (xmin, ymin)]

    crs = {'init':f'epsg:{input_xr.mapping.spatial_epsg}'}
    polygon_geom = Polygon(pts_ls)
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])  

    fig, ax = plt.subplots()
    ax = plt.axes(projection=ccrs.UTM(zone = 45, southern_hemisphere=False))
    input_xr.v.plot(ax=ax)
    polygon.plot(ax=ax, facecolor='none', edgecolor='red') 


    #class cartopy.crs.UTM(zone, southern_hemisphere=False, globe=None)

    return polygon    



First zarr url (getting these from the itslive s3 data cube catalog)

In [3]:
zarr_32645_x350000_y3050000 = 's3://its-live-data/datacubes/v02/N20E080/ITS_LIVE_vel_EPSG32645_G0120_X350000_Y3050000.zarr'

ds_32645_x350000_y3050000 = xr.open_dataset(zarr_32645_x350000_y3050000, engine = 'zarr',
                            storage_options= {'anon':True})

/var/folders/y9/k06gyzdx25z64l_tdt_q2xv9s680sz/T/ipykernel_17703/2145342222.py:3: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  ds_32645_x350000_y3050000 = xr.open_dataset(zarr_32645_x350000_y3050000, engine = 'zarr',


GroupNotFoundError: group not found at path ''

Dang, doesn't look like it's there

Trying with another url for the same region:

In [4]:
zarr_32645_x350000_y4650000 = "s3://its-live-data/datacubes/v02/N40E080/ITS_LIVE_vel_EPSG32645_G0120_X350000_Y4650000.zarr"

ds_32645_x350000_y4650000 = xr.open_dataset(zarr_32645_x350000_y4650000, engine = 'zarr',
                            storage_options={'anon':True})

Cool! This one worked, let's see the bounding box

In [14]:
bbox_x350000_y4650000 = get_bbox(ds_32645_x350000_y4650000)

/Users/emarshall/miniconda3/envs/mynewbook/lib/python3.10/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


: 

: 